<a href="https://colab.research.google.com/github/JoaoMKlGui/LearningPytorch/blob/main/CIFAR10_Classificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Database classificator
A NN that classifies the images from CIFAR-10 database applying CNN's fro Pytorch

The classes that it identifies are:
- Airplane
- Automobile
- Bird
- Cat
- Deer
- Dog
- Frog
- Horse
- Ship
- Truck


In [1]:
# Libraries
!pip install torchmetrics
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchmetrics import Accuracy
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor
from torch.utils.data import Subset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 20.0 MB/s eta 0:00:00


In [2]:
import torch.nn as nn

In [3]:
# GPU / CPU
if torch.cuda.is_available():
   device = "cuda:0"
   print(torch.cuda.get_device_name(device))
else:
   device = "cpu"
print(device)
device = torch.device(device)

Tesla T4
cuda:0


In [4]:
raw_train = datasets.CIFAR10(root='data', train=True, transform=ToTensor(),download=True)
raw_train

#10 categories, coloured 32x32 images, 50000 images for training

100%|██████████| 170498071/170498071 [00:13<00:00, 12900614.91it/s]


Extracting data/cifar-10-python.tar.gz to data


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
raw_test = datasets.CIFAR10(root='data', train=False, transform=ToTensor(), download=True)
raw_test

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

Classes that will be predicted

In [6]:
raw_test.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

Dividing my train data into a train dataset and a validation dataset

In [7]:
train, validation = train_test_split(raw_train, test_size=0.2, stratify = raw_train.targets)

Converting to DataLoader

In [8]:
test_loader = DataLoader(dataset = raw_test)
train_loader = DataLoader(dataset=train, batch_size = 10, shuffle = True)
validation_loader = DataLoader(dataset=validation)

train_loader

In [10]:
#Neural Network

class Classificator(nn.Module):

  #32x32 images
  #lost 2 pixel p/conv layer

  def __init__(self):
    super().__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(3, 32, (3,3)),
        nn.ReLU()
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, (3,3)),
        nn.ReLU()
    )

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, (3,3)),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2))
    )

    self.layer4 = nn.Sequential(
        nn.Conv2d(128, 256, (3,3)),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2,2))
    )

    self.dense_linear = nn.Sequential(
        nn.Linear(256*(5)*(5), 256),
        nn.ReLU()
    )

    self.dropout1 = nn.Dropout(p=0.25)

    self.dropout2 = nn.Dropout(p=0.50)

    self.out_linear = nn.Sequential(
        nn.Linear(256, 10),
        nn.ReLU(),
        nn.Softmax(dim=1)
    )



  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.dropout1(x)
    x = self.dense_linear(x.flatten(1))
    x = self.dropout2(x)
    x = self.out_linear(x)

    return x

Training

In [11]:
model = Classificator().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
loss_function = nn.CrossEntropyLoss()

accuracy = Accuracy(task='multiclass', num_classes= 10).to('cuda')

for i in range(100):

    acuraciaAcumulador = 0
    y_true = []
    y_pred = []

    model.train()
    # training

    for train_x, train_y in train_loader:

      # predict

      train_x, train_y = train_x.to('cuda'), train_y.to('cuda')

      outputs = model(train_x)
      outputs = outputs.squeeze()

      # calculating loss
      batch_loss = loss_function(outputs, train_y)

      # zero, backpropagation, ajusta parâmetros pelo gradiente descendente
      optimizer.zero_grad()
      batch_loss.backward()
      optimizer.step()

    with torch.no_grad():
      model.eval()
      for valid_x, valid_y in validation_loader:

        valid_x, valid_y = valid_x.to('cuda'), valid_y.to('cuda')

        outputs = model(valid_x)
        outputs.squeeze()

        y_true.append(valid_y)
        y_pred.append(torch.argmax(outputs, dim=1))

        acuraciaAcumulador += accuracy(outputs, valid_y)

      acuraciaTotal = acuraciaAcumulador / len(validation_loader)
      print(f'Época: {i+1:d}, Accuracy: {100*acuraciaTotal:.2f}')

Época: 1, Accuracy: 47.88
Época: 2, Accuracy: 54.90
Época: 3, Accuracy: 57.13
Época: 4, Accuracy: 58.94
Época: 5, Accuracy: 61.27
Época: 6, Accuracy: 61.61
Época: 7, Accuracy: 62.93
Época: 8, Accuracy: 63.94
Época: 9, Accuracy: 63.89
Época: 10, Accuracy: 65.02
Época: 11, Accuracy: 65.48
Época: 12, Accuracy: 69.35
Época: 13, Accuracy: 71.41
Época: 14, Accuracy: 70.07
Época: 15, Accuracy: 71.49
Época: 16, Accuracy: 72.42
Época: 17, Accuracy: 73.49
Época: 18, Accuracy: 73.84
Época: 19, Accuracy: 74.01
Época: 20, Accuracy: 73.57
Época: 21, Accuracy: 74.17
Época: 22, Accuracy: 75.50
Época: 23, Accuracy: 74.82
Época: 24, Accuracy: 75.06
Época: 25, Accuracy: 75.06
Época: 26, Accuracy: 75.26
Época: 27, Accuracy: 74.65
Época: 28, Accuracy: 76.15
Época: 29, Accuracy: 76.06
Época: 30, Accuracy: 76.33
Época: 31, Accuracy: 76.60
Época: 32, Accuracy: 76.07
Época: 33, Accuracy: 75.97
Época: 34, Accuracy: 76.66
Época: 35, Accuracy: 77.33
Época: 36, Accuracy: 76.45
Época: 37, Accuracy: 76.74
Época: 38,

In [12]:
from torch import save, load

Saving the model

In [13]:
with open('modelo.pt', 'wb') as f:
  save(model.state_dict(), f)

Testing and evalutating the model with accuracy

In [14]:
with open('modelo.pt', 'rb') as f:
  model.load_state_dict(load(f))
  acumulador = 0
  with torch.no_grad():
    model.eval()

    for test_x, test_y in test_loader:

      test_x, test_y = test_x.to('cuda'), test_y.to('cuda')

      outputs = model(test_x)
      outputs.squeeze()

      acumulador += accuracy(outputs, test_y).item()

    acuracia_teste = acumulador / len(test_loader)
    print('Test Accuracy:', acuracia_teste*100)




Test Accuracy: 80.0
